In [ ]:
# The first part of this script consists of processing the data and obtaining random sampling with replacement counts
# The second part of the script was used for plotting the counts.
# The files generated by this script can be found in the supplementary data. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
file_path = 'PDB_metadata.csv'
PDB_df = pd.read_csv(file_path, sep=',',low_memory=False)

In [ ]:
# Processing phage taxonomy column for bacteria (Prophage-DB)
all_bacteria_df = PDB_df[PDB_df['host_domain'] == 'Bacteria'].copy()
lineage_split = all_bacteria_df['lineage_genomad'].str.split(';')

realm_list = []
kingdom_list = []
phylum_list = []
class_list = []
order_list = []
family_list = []

# Iterate through each row in the DataFrame
for row in lineage_split:
    # Initialize variables to store lineage information for the current row
    realm = None
    kingdom = None
    phylum = None
    class_ = None
    order_ = None
    family = None
    
    # Iterate through each element in the row
    for element in row:
        element = element.strip()
        if element.endswith('viria'):
            realm = element
        elif element.endswith('virae'):
            kingdom = element
        elif element.endswith('viricota'):
            phylum = element
        elif element.endswith('viricetes'):
            class_ = element
        elif element.endswith('virales'):
            order_ = element
        elif element.endswith('viridae'):
            family = element
    
    # Append the lineage information for the current row to the lists
    realm_list.append(realm)
    kingdom_list.append(kingdom)
    phylum_list.append(phylum)
    class_list.append(class_)
    order_list.append(order_)
    family_list.append(family)

# Assign the lists to the DataFrame as new columns
all_bacteria_df['realm_phage'] = realm_list
all_bacteria_df['kingdom_phage'] = kingdom_list
all_bacteria_df['phylum_phage'] = phylum_list
all_bacteria_df['class_phage'] = class_list
all_bacteria_df['order_phage'] = order_list
all_bacteria_df['family_phage'] = family_list

In [ ]:
# Processing phage taxonomy column for archaea (Prophage-DB)
all_archaea_df = PDB_df[PDB_df['host_domain'] == 'Archaea'].copy()
lineage_split = all_archaea_df['lineage_genomad'].str.split(';')

realm_list = []
kingdom_list = []
phylum_list = []
class_list = []
order_list = []
family_list = []

# Iterate through each row in the DataFrame
for row in lineage_split:
    # Initialize variables to store lineage information for the current row
    realm = None
    kingdom = None
    phylum = None
    class_ = None
    order_ = None
    family = None
    
    # Iterate through each element in the row
    for element in row:
        element = element.strip()
        if element.endswith('viria'):
            realm = element
        elif element.endswith('virae'):
            kingdom = element
        elif element.endswith('viricota'):
            phylum = element
        elif element.endswith('viricetes'):
            class_ = element
        elif element.endswith('virales'):
            order_ = element
        elif element.endswith('viridae'):
            family = element
    
    # Append the lineage information for the current row to the lists
    realm_list.append(realm)
    kingdom_list.append(kingdom)
    phylum_list.append(phylum)
    class_list.append(class_)
    order_list.append(order_)
    family_list.append(family)

# Assign the lists to the DataFrame as new columns
all_archaea_df['realm_phage'] = realm_list
all_archaea_df['kingdom_phage'] = kingdom_list
all_archaea_df['phylum_phage'] = phylum_list
all_archaea_df['class_phage'] = class_list
all_archaea_df['order_phage'] = order_list
all_archaea_df['family_phage'] = family_list

In [ ]:
# Function to extract information for each taxonomic group (bacterial phages)
def extract_taxonomic_info(row, group):
    parts = row.split(';')
    for part in parts:
        if part.startswith(group):
            return part.split('__')[1]
    return None

# Select the columns of interest only if they exist in the DataFrame
columns_of_interest = ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']
bacteria_result_df = all_bacteria_df[columns_of_interest].copy()  # Create a copy

# Update specified columns with 'Unclassified' only for the selected row
columns_to_write_unclassified = ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']
bacteria_result_df = bacteria_result_df.fillna('Unclassified')

columns_to_process =  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']
# Define the desired sample size
desired_sample_size = 10000

# DataFrame to store sampled data and word counts for each column
column_results_df = pd.DataFrame()

# Iterate through the columns
for column in columns_to_process:
    column_without_na = bacteria_result_df[column].dropna()
    # Perform random sampling without replacement on the current column
    sampled_data = column_without_na.sample(n=desired_sample_size, replace=True)
    # Reset the index to avoid duplicate labels
    sampled_data = sampled_data.reset_index(drop=True)
    # Get the count of each unique word in the sampled data
    word_counts = sampled_data.value_counts()
    # Add results to the DataFrame
    column_results_df[column + '_sampled_data'] = sampled_data
    column_results_df[column + '_word_counts'] = word_counts

# Exclude 'Taxonomic classification' column from the count
columns_to_count = column_results_df.columns.str.replace('_sampled_data', '')

column_counts = {}

# Order of taxonomic groups
order_of_groups =  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']

for column in order_of_groups:
    if column in columns_to_count:
        counts = column_results_df[column + '_sampled_data'].value_counts()
        column_counts[column] = pd.DataFrame({'Name': counts.index, 'Count': counts.values})

# Create a new DataFrame by concactenating the counts
bacteria_taxonomic_counts_df = pd.concat(column_counts.values(), keys=column_counts.keys(), axis=1)
# Iterate over each level 'r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__'
for level in  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']:
    # Get the counts for the current level
    counts = bacteria_taxonomic_counts_df[level, 'Count']
    # Calculate the total count for the current level
    total_count = counts.sum()
    # Calculate the percentage for each count within the current level
    percentage = counts / total_count
    # Add a new column for the percentage to the original DataFrame using .loc
    bacteria_taxonomic_counts_df.loc[:, (level, 'Percentage')] = percentage
    # Add a new column for the logarithm of the percentage using .loc
    bacteria_taxonomic_counts_df.loc[:, (level, 'Logarithm')] = percentage.apply(lambda x: np.log10(x))

# Reorganize the columns to have 'Name', 'Count', 'Percentage', and 'Logarithm' for each level
new_columns = []
for level in  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']:
    new_columns.extend([(level, 'Name'), (level, 'Count'), (level, 'Percentage'), (level, 'Logarithm')])

# Reorder the DataFrame columns
bacteria_taxonomic_counts_df = bacteria_taxonomic_counts_df[new_columns]

bacteria_taxonomic_counts_df.to_csv('counts_for_bacterial_phages_PDB.csv', index=False)

In [ ]:
# Function to extract information for each taxonomic group (archaeal phages)
def extract_taxonomic_info(row, group):
    parts = row.split(';')
    for part in parts:
        if part.startswith(group):
            return part.split('__')[1]
    return None

# Select the columns of interest only if they exist in the DataFrame
columns_of_interest = ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']
archaea_result_df = all_archaea_df[columns_of_interest].copy()  # Create a copy

# Update specified columns with 'Unclassified' only for the selected row
columns_to_write_unclassified = ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']
archaea_result_df = archaea_result_df.fillna('Unclassified')

columns_to_process =  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']
# Define the desired sample size
desired_sample_size = 10000

# DataFrame to store sampled data and word counts for each column
column_results_df = pd.DataFrame()

# Iterate through the columns
for column in columns_to_process:
    column_without_na = archaea_result_df[column].dropna()
    # Perform random sampling without replacement on the current column
    sampled_data = column_without_na.sample(n=desired_sample_size, replace=True)
    # Reset the index to avoid duplicate labels
    sampled_data = sampled_data.reset_index(drop=True)
    # Get the count of each unique word in the sampled data
    word_counts = sampled_data.value_counts()
    # Add results to the DataFrame
    column_results_df[column + '_sampled_data'] = sampled_data
    column_results_df[column + '_word_counts'] = word_counts

# Exclude 'Taxonomic classification' column from the count
columns_to_count = column_results_df.columns.str.replace('_sampled_data', '')

column_counts = {}

# Order of taxonomic groups
order_of_groups =  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']

for column in order_of_groups:
    if column in columns_to_count:
        counts = column_results_df[column + '_sampled_data'].value_counts()
        column_counts[column] = pd.DataFrame({'Name': counts.index, 'Count': counts.values})

# Create a new DataFrame by concactenating the counts
archaea_taxonomic_counts_df = pd.concat(column_counts.values(), keys=column_counts.keys(), axis=1)
# Iterate over each level 'r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__'
for level in  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']:
    # Get the counts for the current level
    counts = archaea_taxonomic_counts_df[level, 'Count']
    # Calculate the total count for the current level
    total_count = counts.sum()
    # Calculate the percentage for each count within the current level
    percentage = counts / total_count
    # Add a new column for the percentage to the original DataFrame using .loc
    archaea_taxonomic_counts_df.loc[:, (level, 'Percentage')] = percentage
    # Add a new column for the logarithm of the percentage using .loc
    archaea_taxonomic_counts_df.loc[:, (level, 'Logarithm')] = percentage.apply(lambda x: np.log10(x))

# Reorganize the columns to have 'Name', 'Count', 'Percentage', and 'Logarithm' for each level
new_columns = []
for level in  ['realm_phage','kingdom_phage','phylum_phage','class_phage','order_phage','family_phage']:
    new_columns.extend([(level, 'Name'), (level, 'Count'), (level, 'Percentage'), (level, 'Logarithm')])

# Reorder the DataFrame columns
archaea_taxonomic_counts_df = archaea_taxonomic_counts_df[new_columns]

archaea_taxonomic_counts_df.to_csv('counts_for_archaeal_phages_PDB.csv', index=False)

In [ ]:
# This file contains all metadata in IMG/VR 4 that is classified as provirus
file_path = 'imgvr_provirus.csv'
# Read the CSV file into a DataFrame
imgvr_df = pd.read_csv(file_path, sep=',',low_memory=False)

In [ ]:
# Processing phage taxonomy column for bacteria (IMGVR)
def extract_taxonomic_info(row, group):
    parts = row.split(';')
    for part in parts:
        if part.startswith(group):
            return part.split('__')[1]
    return None

all_bacteria_df_copy = all_bacteria_df.copy()

# Apply the function to create new columns for each taxonomic group
for group in ['r', 'k', 'p', 'c', 'o', 'f', 'g', 's']:
    all_bacteria_df_copy[group + '__'] = all_bacteria_df_copy['Taxonomic classification'].apply(lambda x: extract_taxonomic_info(x, group))

# Select the columns of interest only if they exist in the DataFrame
columns_of_interest = ['Taxonomic classification'] + [group + '__' for group in ['r', 'k', 'p', 'c', 'o', 'f', 'g', 's']]
all_bacteria_df_result_df = all_bacteria_df_copy[columns_of_interest].copy()  # Create a copy

# Add 'Unclassified' column based on 'Taxonomic classification'
all_bacteria_df_result_df['Unclassified'] = all_bacteria_df_result_df['Taxonomic classification'].apply(lambda x: 'Unclassified' if ';;;;;;;' in x else '')

# Move 'Unclassified' column next to 's__'
all_bacteria_df_result_df.insert(
    all_bacteria_df_result_df.columns.get_loc('s__') + 1,
    'Unclassified',
    all_bacteria_df_result_df.pop('Unclassified')
)

# Add Unclassified to DataFrame
mask = all_bacteria_df_result_df['Taxonomic classification'] == ';;;;;;;'
columns_to_write_unclassified = ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']
all_bacteria_df_result_df.loc[mask, columns_to_write_unclassified] = 'Unclassified'
all_bacteria_df_result_df = all_bacteria_df_result_df.fillna('Unclassified')

# Random Sampling With Replacement
columns_to_process = ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']

# Define the desired sample size
desired_sample_size = 10000

# DataFrame to store sampled data and word counts for each column
column_results_df = pd.DataFrame()

# Iterate through the columns
for column in columns_to_process:

    column_without_na = all_bacteria_df_result_df[column].dropna()
    # Perform random sampling without replacement on the current column
    sampled_data = column_without_na.sample(n=desired_sample_size, replace=True)
    
    # Reset the index to avoid duplicate labels
    sampled_data = sampled_data.reset_index(drop=True)
    
    # Get the count of each unique word in the sampled data
    word_counts = sampled_data.value_counts()
    
    # Add results to the DataFrame
    column_results_df[column + '_sampled_data'] = sampled_data
    column_results_df[column + '_word_counts'] = word_counts

# Exclude 'Taxonomic classification' column from the count
columns_to_count = column_results_df.columns.str.replace('_sampled_data', '')

column_counts = {}

# Order of taxonomic groups
order_of_groups = ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']

for column in order_of_groups:
    if column in columns_to_count:
        counts = column_results_df[column + '_sampled_data'].value_counts()
        column_counts[column] = pd.DataFrame({'Name': counts.index, 'Count': counts.values})

# Create a new DataFrame by concatenating the counts
all_bacteria_taxonomic_counts_df = pd.concat(column_counts.values(), keys=column_counts.keys(), axis=1)

# Get proportions (percentage) and log (base10)
for level in ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']:
    # Get the counts for the current level
    counts = all_bacteria_taxonomic_counts_df[level, 'Count']
    # Calculate the total count for the current level
    total_count = counts.sum()
    # Calculate the percentage for each count within the current level
    percentage = counts / total_count
    # Add a new column for the percentage to the original DataFrame using .loc
    all_bacteria_taxonomic_counts_df.loc[:, (level, 'Percentage')] = percentage
    # Add a new column for the logarithm of the percentage using .loc
    all_bacteria_taxonomic_counts_df.loc[:, (level, 'Logarithm')] = percentage.apply(lambda x: np.log10(x))

# Reorganize the columns to have 'Name', 'Count', 'Percentage', and 'Logarithm' for each level
new_columns = []
for level in ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']:
    new_columns.extend([(level, 'Name'), (level, 'Count'), (level, 'Percentage'), (level, 'Logarithm')])

# Reorder the DataFrame columns
all_bacteria_taxonomic_counts_df = all_bacteria_taxonomic_counts_df[new_columns]

all_bacteria_taxonomic_counts_df.to_csv('counts_for_bacterial_phages_IMGVR.csv', index=False)

In [ ]:
# Processing phage taxonomy column for archaea (IMGVR)
def extract_taxonomic_info(row, group):
    parts = row.split(';')
    for part in parts:
        if part.startswith(group):
            return part.split('__')[1]
    return None

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
all_archaea_df_copy = all_archaea_df.copy()

# Apply the function to create new columns for each taxonomic group
for group in ['r', 'k', 'p', 'c', 'o', 'f', 'g', 's']:
    all_archaea_df_copy[group + '__'] = all_archaea_df_copy['Taxonomic classification'].apply(lambda x: extract_taxonomic_info(x, group))

# Select the columns of interest only if they exist in the DataFrame
columns_of_interest = ['Taxonomic classification'] + [group + '__' for group in ['r', 'k', 'p', 'c', 'o', 'f', 'g', 's']]
all_archaea_df_result_df = all_archaea_df_copy[columns_of_interest].copy()  # Create a copy

# Add 'Unclassified' column based on 'Taxonomic classification'
all_archaea_df_result_df['Unclassified'] = all_archaea_df_result_df['Taxonomic classification'].apply(lambda x: 'Unclassified' if ';;;;;;;' in x else '')

# Move 'Unclassified' column next to 's__'
all_archaea_df_result_df.insert(
    all_archaea_df_result_df.columns.get_loc('s__') + 1,
    'Unclassified',
    all_archaea_df_result_df.pop('Unclassified')
)

# Add Unclassified to DataFrame
mask = all_archaea_df_result_df['Taxonomic classification'] == ';;;;;;;'
columns_to_write_unclassified = ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']
all_archaea_df_result_df.loc[mask, columns_to_write_unclassified] = 'Unclassified'
all_archaea_df_result_df = all_archaea_df_result_df.fillna('Unclassified')

# Random Sampling With Replacement
columns_to_process = ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']

# Define the desired sample size
desired_sample_size = 10000

# DataFrame to store sampled data and word counts for each column
column_results_df = pd.DataFrame()

# Iterate through the columns
for column in columns_to_process:

    column_without_na = all_archaea_df_result_df[column].dropna()
    # Perform random sampling without replacement on the current column
    sampled_data = column_without_na.sample(n=desired_sample_size, replace=True)
    
    # Reset the index to avoid duplicate labels
    sampled_data = sampled_data.reset_index(drop=True)
    
    # Get the count of each unique word in the sampled data
    word_counts = sampled_data.value_counts()
    
    # Add results to the DataFrame
    column_results_df[column + '_sampled_data'] = sampled_data
    column_results_df[column + '_word_counts'] = word_counts

# Exclude 'Taxonomic classification' column from the count
columns_to_count = column_results_df.columns.str.replace('_sampled_data', '')

column_counts = {}

# Order of taxonomic groups
order_of_groups = ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']

for column in order_of_groups:
    if column in columns_to_count:
        counts = column_results_df[column + '_sampled_data'].value_counts()
        column_counts[column] = pd.DataFrame({'Name': counts.index, 'Count': counts.values})

# Create a new DataFrame by concatenating the counts
all_archaea_taxonomic_counts_df = pd.concat(column_counts.values(), keys=column_counts.keys(), axis=1)

# Get proportions (percentage) and log (base10)
for level in ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']:
    # Get the counts for the current level
    counts = all_archaea_taxonomic_counts_df[level, 'Count']
    # Calculate the total count for the current level
    total_count = counts.sum()
    # Calculate the percentage for each count within the current level
    percentage = counts / total_count
    # Add a new column for the percentage to the original DataFrame using .loc
    all_archaea_taxonomic_counts_df.loc[:, (level, 'Percentage')] = percentage
    # Add a new column for the logarithm of the percentage using .loc
    all_archaea_taxonomic_counts_df.loc[:, (level, 'Logarithm')] = percentage.apply(lambda x: np.log10(x))

# Reorganize the columns to have 'Name', 'Count', 'Percentage', and 'Logarithm' for each level
new_columns = []
for level in ['r__', 'k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']:
    new_columns.extend([(level, 'Name'), (level, 'Count'), (level, 'Percentage'), (level, 'Logarithm')])

# Reorder the DataFrame columns
all_archaea_taxonomic_counts_df = all_archaea_taxonomic_counts_df[new_columns]

# Display the updated DataFrame
all_archaea_taxonomic_counts_df
all_archaea_taxonomic_counts_df.to_csv('counts_for_archaeal_phages_IMGVR.csv', index=False)


In [ ]:
# Plotting bacterial phages
def read_and_convert_data(file1, file2, pdb_column, imgvr_column):
    # Read data from files
    pdb_data = pd.read_csv(file1)
    imgvr_data = pd.read_csv(file2)
    
    # Extract required columns
    pdb_data = pdb_data[[pdb_column + '_PDB', pdb_column + '_percentage_PDB']]
    imgvr_data = imgvr_data[[imgvr_column + '_IMGVR', imgvr_column + '_percentage_IMGVR']]
    
    # Remove rows with missing values (NaN)
    pdb_data.dropna(inplace=True)
    imgvr_data.dropna(inplace=True)
    
    return pdb_data, imgvr_data

def plot_combined_counts(ax, data, border_width=1.5):
    y_pos = 0
    y_ticks = []
    y_labels = []
    label_colors = []

    # Colors for observed and expected counts
    observed_color = '#8ac926'
    expected_color = '#008bf8'

    # Colors for y-axis labels based on columns
    column_colors = {
        'Realm': '#ff6b6b',
        'Kingdom': '#6A5ACD',
        'Phylum': '#FFA500',
        'Class': '#A0B4FF',
        'Order': '#6bd425',
        'Family': '#00BFFF'
    }

    # Plot each column's data
    for column_name, pdb_data, imgvr_data in data:
        # Merge dataframes on specified column with an outer join to ensure all names are included
        merged_data = pd.merge(pdb_data, imgvr_data, how='outer', left_on=column_name + '_PDB', right_on=column_name + '_IMGVR')
        
        # Fill missing percentages with 0
        merged_data[column_name + '_percentage_PDB'].fillna(0, inplace=True)
        merged_data[column_name + '_percentage_IMGVR'].fillna(0, inplace=True)
        
        # Filter out rows where category values are NaN
        merged_data = merged_data[~merged_data[column_name + '_PDB'].isna()]
        
        # Extract observed and expected counts from the merged dataframe
        observed_col = column_name + '_percentage_PDB'
        expected_col = column_name + '_percentage_IMGVR'
        observed_counts = merged_data[observed_col].values
        expected_counts = merged_data[expected_col].values
        
        # Calculate the standard deviation of each group
        observed_std = np.std(observed_counts, ddof=1)
        expected_std = np.std(expected_counts, ddof=1)

        # Calculate the sample sizes of each group
        observed_n = len(observed_counts)
        expected_n = len(expected_counts)

        # Calculate the standard error for each group
        observed_se = observed_std / np.sqrt(observed_n)
        expected_se = expected_std / np.sqrt(expected_n)

        # Extract column names
        column_names = merged_data[column_name + '_PDB'].values

        # Create a horizontal bar graph with error bars and outlines
        y = np.arange(y_pos, y_pos + len(observed_counts) * 2, 2)
        height = 0.8  # Adjust the height to make the bars slightly smaller than the gap
        ax.barh(y - height/2, observed_counts, height, edgecolor='black', color=observed_color, linewidth=border_width, label='Observed')
        ax.barh(y + height/2, expected_counts, height, edgecolor='black', color=expected_color, linewidth=border_width, label='Expected')
        ax.errorbar(observed_counts, y - height/2, xerr=observed_se, fmt='none', capsize=3, ecolor='black')
        ax.errorbar(expected_counts, y + height/2, xerr=expected_se, fmt='none', capsize=3, ecolor='black')

        # Append the current y positions and labels for setting y-ticks later
        y_ticks.extend(y)
        y_labels.extend(column_names)
        label_colors.extend([column_colors[column_name]] * len(column_names))
        y_pos += len(observed_counts) * 2  # Increase y_pos by double the number of counts to create separation

    # Set y-ticks without labels
    ax.set_yticks(y_ticks)
    ax.set_yticklabels([])

    # Add colored labels manually
    for y, label, color in zip(y_ticks, y_labels, label_colors):
        ax.text(-0.02, y, label, ha='right', va='center', fontsize=20, fontweight='bold', color=color, transform=ax.get_yaxis_transform())

    # Set y-axis limits to fit the bars without extra space
    ax.set_ylim(min(y_ticks) - 1, max(y_ticks) + 1)

    # Customize x-axis ticks to remove the -0.2 value
    x_ticks = ax.get_xticks()
    x_ticks = x_ticks[x_ticks >= 0]
    x_ticks = x_ticks[x_ticks <= 1.0]
    ax.set_xticks(x_ticks)

    # Unique legend handles and labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), prop={'size': 20, 'weight': 'bold'}, loc='upper right')
    
    ax.tick_params(axis='x', labelsize=20)
    for label in ax.get_xticklabels():
        label.set_fontweight('bold')

file1 = "counts_for_bacterial_phages_PDB.csv"
file2 = "counts_for_bacterial_phages_IMGVR.csv"

# Assuming the column names in your dataframes are 'Kingdom_PDB', 'Kingdom_IMGVR', 'Realm_PDB', 'Realm_IMGVR', etc.
columns = ['Realm', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family']

data = []
for column in columns:
    pdb_data, imgvr_data = read_and_convert_data(file1, file2, pdb_column=column, imgvr_column=column)
    data.append((column, pdb_data, imgvr_data))

# Create a single figure with one set of axes
fig, ax = plt.subplots(figsize=(10, 25))

# Plot combined data
plot_combined_counts(ax, data, border_width=0.01)

# Add a title to the figure, centered above the bars
ax.text(0.5, 1.02, 'Bacterial phages', fontsize=30, fontweight='bold', ha='center', transform=ax.transAxes)

# Adjust the layout 
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.subplots_adjust(top=0.95, bottom=0.05)

plt.show()


In [ ]:
# Plotting archaeal phages

def read_and_convert_data(file1, file2, pdb_column, imgvr_column):
    # Read data from files
    pdb_data = pd.read_csv(file1)
    imgvr_data = pd.read_csv(file2)
    
    # Extract required columns
    pdb_data = pdb_data[[pdb_column + '_PDB', pdb_column + '_percentage_PDB']]
    imgvr_data = imgvr_data[[imgvr_column + '_IMGVR', imgvr_column + '_percentage_IMGVR']]
    
    # Remove rows with missing values (NaN)
    pdb_data.dropna(inplace=True)
    imgvr_data.dropna(inplace=True)
    
    return pdb_data, imgvr_data

def plot_combined_counts(ax, data, border_width=1.5):
    y_pos = 0
    y_ticks = []
    y_labels = []
    label_colors = []

    # Colors for observed and expected counts
    observed_color = '#ff6b6b'
    expected_color = '#008bf8'

    # Colors for y-axis labels based on columns
    column_colors = {
        'Realm': '#ff6b6b',
        'Kingdom': '#6A5ACD',
        'Phylum': '#FFA500',
        'Class': '#A0B4FF',
        'Order': '#6bd425',
        'Family': '#00BFFF'
    }

    # Plot each column's data
    for column_name, pdb_data, imgvr_data in data:
        # Merge dataframes on specified column with an outer join to ensure all names are included
        merged_data = pd.merge(pdb_data, imgvr_data, how='outer', left_on=column_name + '_PDB', right_on=column_name + '_IMGVR')
        
        # Fill missing percentages with 0
        merged_data[column_name + '_percentage_PDB'].fillna(0, inplace=True)
        merged_data[column_name + '_percentage_IMGVR'].fillna(0, inplace=True)
        
        # Filter out rows where category values are NaN
        merged_data = merged_data[~merged_data[column_name + '_PDB'].isna()]
        
        # Extract observed and expected counts from the merged dataframe
        observed_col = column_name + '_percentage_PDB'
        expected_col = column_name + '_percentage_IMGVR'
        observed_counts = merged_data[observed_col].values
        expected_counts = merged_data[expected_col].values
        
        # Calculate the standard deviation of each group
        observed_std = np.std(observed_counts, ddof=1)
        expected_std = np.std(expected_counts, ddof=1)

        # Calculate the sample sizes of each group
        observed_n = len(observed_counts)
        expected_n = len(expected_counts)

        # Calculate the standard error for each group
        observed_se = observed_std / np.sqrt(observed_n)
        expected_se = expected_std / np.sqrt(expected_n)

        # Extract column names
        column_names = merged_data[column_name + '_PDB'].values

        # Create a horizontal bar graph with error bars and outlines
        y = np.arange(y_pos, y_pos + len(observed_counts) * 2, 2)
        height = 0.8  # Adjust the height to make the bars slightly smaller than the gap
        ax.barh(y - height/2, observed_counts, height, edgecolor='black', color=observed_color, linewidth=border_width, label='Observed')
        ax.barh(y + height/2, expected_counts, height, edgecolor='black', color=expected_color, linewidth=border_width, label='Expected')
        ax.errorbar(observed_counts, y - height/2, xerr=observed_se, fmt='none', capsize=3, ecolor='black')
        ax.errorbar(expected_counts, y + height/2, xerr=expected_se, fmt='none', capsize=3, ecolor='black')

        # Append the current y positions and labels for setting y-ticks later
        y_ticks.extend(y)
        y_labels.extend(column_names)
        label_colors.extend([column_colors[column_name]] * len(column_names))
        y_pos += len(observed_counts) * 2  # Increase y_pos by double the number of counts to create separation

    # Set y-ticks without labels
    ax.set_yticks(y_ticks)
    ax.set_yticklabels([])

    # Add colored labels manually
    for y, label, color in zip(y_ticks, y_labels, label_colors):
        ax.text(-0.02, y, label, ha='right', va='center', fontsize=20, fontweight='bold', color=color, transform=ax.get_yaxis_transform())

    # Set y-axis limits to fit the bars without extra space
    ax.set_ylim(min(y_ticks) - 1, max(y_ticks) + 1)

    # Customize x-axis ticks to remove the -0.2 value
    x_ticks = ax.get_xticks()
    x_ticks = x_ticks[x_ticks >= 0]
    x_ticks = x_ticks[x_ticks <= 1.0]
    ax.set_xticks(x_ticks)

    # Get unique legend handles and labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), prop={'size': 20, 'weight': 'bold'}, loc='upper right')
    
    ax.tick_params(axis='x', labelsize=20)
    for label in ax.get_xticklabels():
        label.set_fontweight('bold')

file1 = "counts_for_archaeal_phages_PDB.csv"
file2 = "counts_for_archaeal_phages_IMGVR.csv"

# Assuming the column names in your dataframes are 'Kingdom_PDB', 'Kingdom_IMGVR', 'Realm_PDB', 'Realm_IMGVR', etc.
columns = ['Realm', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family']

data = []
for column in columns:
    pdb_data, imgvr_data = read_and_convert_data(file1, file2, pdb_column=column, imgvr_column=column)
    data.append((column, pdb_data, imgvr_data))

# Create a single figure with one set of axes
fig, ax = plt.subplots(figsize=(10, 25))

# Plot combined data
plot_combined_counts(ax, data, border_width=0.01)

# Add a title to the figure, centered above the bars
ax.text(0.5, 1.02, 'Archaeal phages', fontsize=30, fontweight='bold', ha='center', transform=ax.transAxes)

# Adjust the layout 
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.subplots_adjust(top=0.95, bottom=0.05)

plt.show()